In [80]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [81]:
mvps = pd.read_csv('../data/raw/1981_2024_mvps.csv')
standings = pd.read_csv('../data/raw/1981_2024_standings.csv')
advanced = pd.read_csv('../data/raw/1981_2024_advanced.csv')
per_game = pd.read_csv('../data/raw/1981_2024_per_game.csv')
print(standings.head(5))

   Year                Team     W     L   W/L%   GB   PS/G   PA/G   SRS  seed
0  1981      Boston Celtics  62.0  20.0  0.756    —  109.9  104.0  6.05     1
1  1981  Philadelphia 76ers  62.0  20.0  0.756    —  111.7  103.8  7.76     2
2  1981     Milwaukee Bucks  60.0  22.0  0.732    —  113.1  105.9  7.14     3
3  1981        Phoenix Suns  57.0  25.0  0.695    —  110.0  104.5  4.83     4
4  1981  Los Angeles Lakers  54.0  28.0  0.659  3.0  111.2  107.3  3.27     5


There is Divisions values inside standings so we want to remove that

In [82]:
standings = standings[standings.Team.str.contains('Division') == False]
print(standings.head(5))

   Year                Team     W     L   W/L%   GB   PS/G   PA/G   SRS  seed
0  1981      Boston Celtics  62.0  20.0  0.756    —  109.9  104.0  6.05     1
1  1981  Philadelphia 76ers  62.0  20.0  0.756    —  111.7  103.8  7.76     2
2  1981     Milwaukee Bucks  60.0  22.0  0.732    —  113.1  105.9  7.14     3
3  1981        Phoenix Suns  57.0  25.0  0.695    —  110.0  104.5  4.83     4
4  1981  Los Angeles Lakers  54.0  28.0  0.659  3.0  111.2  107.3  3.27     5


We only want Win percentage and we only want to get the abbreviation of each teams to correlate what we have with our MVPs datas

In [83]:
standings = standings.drop(columns=["W","L",'GB', 'PS/G', 'PA/G', 'SRS'])
print(standings.head(5))

   Year                Team   W/L%  seed
0  1981      Boston Celtics  0.756     1
1  1981  Philadelphia 76ers  0.756     2
2  1981     Milwaukee Bucks  0.732     3
3  1981        Phoenix Suns  0.695     4
4  1981  Los Angeles Lakers  0.659     5


In [84]:
team_map = {
    'Atlanta Hawks': 'ATL',
    'Boston Celtics': 'BOS',
    'Brooklyn Nets': 'BRK',
    'Charlotte Hornets': 'CHH',
    'Chicago Bulls': 'CHI',
    'Cleveland Cavaliers': 'CLE',
    'Dallas Mavericks': 'DAL',
    'Denver Nuggets': 'DEN',
    'Detroit Pistons': 'DET',
    'Golden State Warriors': 'GSW',
    'Houston Rockets': 'HOU',
    'Indiana Pacers': 'IND',
    'Los Angeles Clippers': 'LAC',
    'Los Angeles Lakers': 'LAL',
    'Memphis Grizzlies': 'MEM',
    'Miami Heat': 'MIA',
    'Milwaukee Bucks': 'MIL',
    'Minnesota Timberwolves': 'MIN',
    'New Orleans Pelicans': 'NOP',
    'New York Knicks': 'NYK',
    'Oklahoma City Thunder': 'OKC',
    'Orlando Magic': 'ORL',
    'Philadelphia 76ers': 'PHI',
    'Phoenix Suns': 'PHO',
    'Portland Trail Blazers': 'POR',
    'Sacramento Kings': 'SAC',
    'San Antonio Spurs': 'SAS',
    'Toronto Raptors': 'TOR',
    'Utah Jazz': 'UTA',
    'Washington Wizards': 'WAS',
    'Washington Bullets': 'WSB',
    "New Jersey Nets": "NJN",
    "Kansas City Kings": "KCK",
    "San Diego Clippers": "SDC",
    "Seattle SuperSonics": "SEA",
    "Vancouver Grizzlies": "VAN",
    "New Orleans Hornets": "NOH",
    "Charlotte Bobcats": "CHA",
    "New Orleans/Oklahoma City Hornets": "NOK",
}
standings['Team'] = standings['Team'].map(team_map)
print(standings.head(5))

   Year Team   W/L%  seed
0  1981  BOS  0.756     1
1  1981  PHI  0.756     2
2  1981  MIL  0.732     3
3  1981  PHO  0.695     4
4  1981  LAL  0.659     5


For each player in the MVPs datas we add the W/L% for their corresponding season and remove all non important or redondant columns

In [85]:
mvps = mvps.drop(columns=["Age", "First", "Pts Won", "Pts Max", "WS", "WS/48", "G", "MP", "PTS", "TRB", "AST", "STL", "BLK", "FG%", "3P%", "FT%", "Tm"])
print(mvps.head(5))

   Year               Player  Share
0  1981        Julius Erving  0.658
1  1981           Larry Bird  0.613
2  1981  Kareem Abdul-Jabbar  0.414
3  1981         Moses Malone  0.261
4  1981        George Gervin  0.120


In [86]:
advanced = pd.merge(advanced, standings[['Year', 'Team', 'W/L%', "seed"]], left_on=['Year', 'Team'], right_on=["Year", "Team"], how='left')
advanced = advanced.drop(columns=["MP"])
per_game = per_game.drop(columns=["Team", "Age", "G", "GS", "Pos", "Awards"])
advanced = pd.merge(advanced, per_game, left_on=['Year', 'Player'], right_on=["Year", "Player"], how='left')

We will need the advanced stats as well so we clean that df too and merge it with the mvps

In [87]:
advanced = advanced.drop_duplicates(subset=['Player', 'Year'], keep='first')
advanced = advanced[advanced.Team.str.contains('TOT') == False]
advanced = advanced[advanced.Team.str.contains('TM') == False]
advanced = advanced.drop(columns=["Pos", "WS", "Awards", "GS"])
advanced = advanced[advanced.Player.str.contains('League') == False]
mvps = pd.merge(advanced, mvps, on=['Year', 'Player'], how="left")
mvps.fillna(0, inplace=True)
mvps = mvps[mvps["G"] > 48]
mvps = mvps[mvps["PER"] > 18]
mvps = mvps[mvps["W/L%"] > 0.5]
mvps = mvps[mvps["MP"] > 30]
print(mvps.head(5))

    Year               Player   Age Team     G   PER    TS%   3PAr    FTr  \
2   1981           Larry Bird  24.0  BOS  82.0  19.9  0.528  0.049  0.218   
8   1981  Kareem Abdul-Jabbar  33.0  LAL  80.0  25.5  0.616  0.001  0.379   
12  1981      Bill Cartwright  23.0  NYK  82.0  18.4  0.611  0.001  0.463   
18  1981        Julius Erving  30.0  PHI  82.0  25.1  0.572  0.012  0.352   
21  1981        Artis Gilmore  31.0  CHI  82.0  21.7  0.699  0.000  0.652   

    ORB%  ...  ORB  DRB   TRB  AST  STL  BLK  TOV   PF   PTS  Share  
2    7.0  ...  2.3  8.6  10.9  5.5  2.0  0.8  3.5  2.9  21.2  0.613  
8    7.6  ...  2.5  7.8  10.3  3.4  0.7  2.9  3.1  3.1  26.2  0.414  
12   6.1  ...  2.0  5.5   7.5  1.4  0.6  1.0  2.4  3.2  20.1  0.000  
18  10.0  ...  3.0  5.0   8.0  4.4  2.1  1.8  3.2  2.8  24.6  0.658  
21   9.2  ...  2.7  7.4  10.1  2.1  0.6  2.4  2.9  3.6  17.9  0.006  

[5 rows x 50 columns]


Now we want to scale our data to have so we use StandardScaler

In [88]:
scaler = StandardScaler()
mvps_scaled = mvps.groupby('Year').apply(lambda x: pd.DataFrame(scaler.fit_transform(x.drop(columns=["Player", "Year", "Team"])), columns=x.columns.drop(["Player", "Year", "Team"])))
mvps_scaled = mvps_scaled.reset_index(drop=True)
mvps_scaled[["Player", "Year", "Team"]] = mvps[["Player", "Year", "Team"]].reset_index(drop=True)
print(mvps_scaled.head(5))
mvps_scaled.to_csv("../data/processed/mvps.csv", index=False)

        Age         G       PER       TS%      3PAr       FTr      ORB%  \
0 -0.815693  0.614919 -0.597857 -1.110137  0.879704 -1.443835 -0.129735   
1  1.725503 -0.391312  1.585619  0.617228 -0.871596 -0.118032  0.174860   
2 -1.098048  0.614919 -1.182716  0.519082 -0.871596  0.573691 -0.586626   
3  0.878438  0.614919  1.429657 -0.246455 -0.470257 -0.340372  1.393236   
4  1.160793  0.614919  0.103975  2.246446 -0.908082  2.130068  0.987111   

       DRB%      TRB%      AST%  ...       AST       STL       BLK       TOV  \
0  1.342074  1.116741  0.639205  ...  1.122679  0.992746 -0.449719  1.557757   
1  0.967163  0.928627 -0.521607  ... -0.371591 -1.081384  1.911306  0.338643   
2  0.445547  0.122424 -1.887268  ... -1.794705 -1.240932 -0.224860 -1.794806   
3 -0.157571  0.256791  0.536781  ...  0.339966  1.152294  0.674579  0.643421   
4  1.293173  1.385476 -1.323932  ... -1.296615 -1.240932  1.349157 -0.270914   

         PF       PTS     Share               Player  Year  Team  
0

C:\Users\Samuel\AppData\Local\Temp\ipykernel_65724\4056481021.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mvps_scaled = mvps.groupby('Year').apply(lambda x: pd.DataFrame(scaler.fit_transform(x.drop(columns=["Player", "Year", "Team"])), columns=x.columns.drop(["Player", "Year", "Team"])))
